In [1]:
import numpy as np
import pandas as pd
import math
from helper.loader import load_ratings, load_movies, load_lists, load_500_1000_corr
from helper.preprocesser import preprocess_ratings, create_fake_identities
from helper.application import application
import helper.collaborative as coll
import helper.showResults as sh

from tqdm import tqdm

## Complete version

In [3]:
application()

100%|██████████| 2314/2314 [00:02<00:00, 961.30it/s]


## Preprocessing cf Jonas

In [4]:
ratings = load_ratings()
movies = load_movies()
lists = load_lists()

In [5]:
ratings_new, lists_new = preprocess_ratings(ratings, lists, min_ratings_user=500, min_ratings_movie=1000)
print(f'{round((len(ratings) - len(ratings_new)) / len(ratings) * 100, 2)} % of ratings removed (out of {len(ratings)})')
print(f'{round((len(lists) - len(lists_new)) / len(lists) * 100, 2)} % lists removed (out of {len(lists)})')

79.36 % of ratings removed (out of 15459945)
68.32 % lists removed (out of 80311)


In [6]:
print("ratings new len {}, index user_id from {} to {}, movie_id from {} to {}".format(len(ratings_new),ratings_new["user_id"].min(),np.max(ratings_new["user_id"]),ratings_new["movie_id"].min(),np.max(ratings_new["movie_id"])))

ratings new len 3190249, index user_id from 2941 to 99988538, movie_id from 10 to 243910


In [7]:
print("{} unique users rate {} unique movies".format(ratings_new["user_id"].nunique(),ratings_new["movie_id"].nunique()))

3900 unique users rate 2941 unique movies


In [8]:
print("{} unique users created {} unique list".format(lists_new["user_id"].nunique(),lists_new["list_id"].nunique()))

1997 unique users created 25442 unique list


## User-item matrix 

The sparse version:

- faster
- lighter in memory
- cleaner
- no index/columns naming possibilities

The dense version:

- DataFrame so more used
- way more data...
- manipulate with care because that thing is killing kernel after kernel... :D

In [9]:
dense_user_item = coll.get_dense_user_item(ratings_new)

dense_user_item

,10,12,13,21,30,44,50,55,61,75,...,229011,232146,233827,234212,234623,234658,235002,235016,241947,243910
2941,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,...,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15344,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29237,NaN,NaN,4.0,NaN,4.0,NaN,NaN,3.0,NaN,5.0,...,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78410,NaN,NaN,NaN,5.0,4.0,3.0,NaN,NaN,4.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83575,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99845748,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99869481,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,4.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99909118,NaN,4.0,5.0,NaN,NaN,NaN,5.0,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99927639,NaN,3.0,NaN,NaN,NaN,NaN,4.0,2.0,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
sparse_user_item,user_id,movie_id = coll.get_sparse_user_item(ratings_new)

In [9]:
print(user_id)
print(movie_id)
sparse_user_item

[    2941    15344    29237 ... 99909118 99927639 99988538]
[    10     12     13 ... 235016 241947 243910]


<3900x2941 sparse matrix of type '<class 'numpy.float64'>'
	with 3190233 stored elements in Compressed Sparse Row format>

## Correlation

### Approach

Static (compute all the correlation once):

- long process
- memory consumption during matrix 
- not realistic (the ratings may change)
- extremely fast at inference (lookup table)

Dynamic (when asked, compute all the correlation between one of the user and the others):

- slower at inference
- must be computed each time we want a recommendation
- the user-item matrix can be modified with new ratings -> changes taken into account

### Implementation

Sparse:

- no correlation (pearsonr consider the 0 as a value of 0 instead of ignoring)

Pandas:

- corr is extremely slow 

Numpy:

- doesn't work with NaN, if replaced by 0, consider it as a value of 0 instead of ignoring it (impact the rating average and therefore the Pearson correlation coefficient)

### Decision

- Ran once the correlation with pandas to get the complete user-user correlation matrix which is saved and now available (WARNING: replace all the diagonal values of 1 with 0 to avoid taking into account itself). Extremely fast (basically a value look up)
- Implement the user-users correlation if we want to run for a specific user on a modified user-item matrix (still quite fast ~2sec for gaining the possibility to change the ratings and adapt the corr)

In [36]:
#corr_matrix_to_save = dense_user_item.T.corr(method="pearson")

In [42]:
#corr_matrix_to_save

,2941,15344,29237,78410,83575,94978,99812,114207,118841,182362,...,99740013,99809019,99817202,99827133,99842199,99845748,99869481,99909118,99927639,99988538
2941,0.000000,0.428091,0.313774,0.412717,0.140430,0.307555,0.333496,0.259837,0.249952,0.371869,...,0.359474,0.151826,0.425525,0.375087,0.277096,0.131071,0.230549,0.284602,0.434867,0.379136
15344,0.428091,0.000000,0.320105,0.389232,-0.040394,0.217663,0.283636,0.188385,0.227881,0.429392,...,0.182316,0.263529,0.424636,0.409913,0.352801,0.197683,0.253783,0.158278,0.438327,0.367678
29237,0.313774,0.320105,0.000000,0.312213,0.036993,0.289640,0.186558,0.135187,0.069235,0.397600,...,0.159255,0.186995,0.381735,0.374622,0.287239,0.050324,0.160204,0.258161,0.423712,0.333702
78410,0.412717,0.389232,0.312213,0.000000,0.072442,0.406765,0.240814,0.119701,0.340255,0.489007,...,0.302066,0.130979,0.478134,0.439999,0.377262,0.224233,0.269535,0.304163,0.477438,0.464281
83575,0.140430,-0.040394,0.036993,0.072442,0.000000,0.169615,0.146804,0.122561,0.177234,0.198552,...,0.290054,0.125784,0.189389,0.043521,0.169425,-0.057015,0.110793,0.073143,0.253698,0.154404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99845748,0.131071,0.197683,0.050324,0.224233,-0.057015,0.192945,0.236315,0.120183,0.227178,0.165379,...,0.238232,0.298568,0.430672,0.216807,0.253668,0.000000,0.152544,0.068944,0.345413,0.227731
99869481,0.230549,0.253783,0.160204,0.269535,0.110793,0.135921,0.115787,0.134768,0.271075,0.211613,...,0.090470,0.097052,0.383331,0.276408,0.092313,0.152544,0.000000,0.025657,0.257208,0.206810
99909118,0.284602,0.158278,0.258161,0.304163,0.073143,0.401549,0.425944,0.134234,0.024645,0.367457,...,0.362264,0.240941,0.222494,0.238388,0.330457,0.068944,0.025657,0.000000,0.303124,0.319565
99927639,0.434867,0.438327,0.423712,0.477438,0.253698,0.463754,0.427678,0.220933,0.347487,0.534573,...,0.288457,0.438649,0.583691,0.552196,0.352156,0.345413,0.257208,0.303124,0.000000,0.549119


In [38]:
#np.fill_diagonal(corr_matrix_to_save.values,0)
#corr_matrix_to_save

,2941,15344,29237,78410,83575,94978,99812,114207,118841,182362,...,99740013,99809019,99817202,99827133,99842199,99845748,99869481,99909118,99927639,99988538
2941,0.000000,0.428091,0.313774,0.412717,0.140430,0.307555,0.333496,0.259837,0.249952,0.371869,...,0.359474,0.151826,0.425525,0.375087,0.277096,0.131071,0.230549,0.284602,0.434867,0.379136
15344,0.428091,0.000000,0.320105,0.389232,-0.040394,0.217663,0.283636,0.188385,0.227881,0.429392,...,0.182316,0.263529,0.424636,0.409913,0.352801,0.197683,0.253783,0.158278,0.438327,0.367678
29237,0.313774,0.320105,0.000000,0.312213,0.036993,0.289640,0.186558,0.135187,0.069235,0.397600,...,0.159255,0.186995,0.381735,0.374622,0.287239,0.050324,0.160204,0.258161,0.423712,0.333702
78410,0.412717,0.389232,0.312213,0.000000,0.072442,0.406765,0.240814,0.119701,0.340255,0.489007,...,0.302066,0.130979,0.478134,0.439999,0.377262,0.224233,0.269535,0.304163,0.477438,0.464281
83575,0.140430,-0.040394,0.036993,0.072442,0.000000,0.169615,0.146804,0.122561,0.177234,0.198552,...,0.290054,0.125784,0.189389,0.043521,0.169425,-0.057015,0.110793,0.073143,0.253698,0.154404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99845748,0.131071,0.197683,0.050324,0.224233,-0.057015,0.192945,0.236315,0.120183,0.227178,0.165379,...,0.238232,0.298568,0.430672,0.216807,0.253668,0.000000,0.152544,0.068944,0.345413,0.227731
99869481,0.230549,0.253783,0.160204,0.269535,0.110793,0.135921,0.115787,0.134768,0.271075,0.211613,...,0.090470,0.097052,0.383331,0.276408,0.092313,0.152544,0.000000,0.025657,0.257208,0.206810
99909118,0.284602,0.158278,0.258161,0.304163,0.073143,0.401549,0.425944,0.134234,0.024645,0.367457,...,0.362264,0.240941,0.222494,0.238388,0.330457,0.068944,0.025657,0.000000,0.303124,0.319565
99927639,0.434867,0.438327,0.423712,0.477438,0.253698,0.463754,0.427678,0.220933,0.347487,0.534573,...,0.288457,0.438649,0.583691,0.552196,0.352156,0.345413,0.257208,0.303124,0.000000,0.549119


In [40]:
#corr_matrix_to_save.columns = corr_matrix_to_save.columns.astype(str)
#mid = int(corr_matrix_to_save.shape[0]/2)


In [49]:
#split1 = corr_matrix_to_save.iloc[:mid,:]
#split2 = corr_matrix_to_save.iloc[mid:,:]

In [50]:
#split1.to_parquet("data/small_user_user_0.parquet")
#split2.to_parquet("data/small_user_user_1.parquet")

In [10]:
correlation_matrix = load_500_1000_corr()

In [11]:
correlation_matrix

,2941,15344,29237,78410,83575,94978,99812,114207,118841,182362,...,99740013,99809019,99817202,99827133,99842199,99845748,99869481,99909118,99927639,99988538
2941,0.000000,0.428091,0.313774,0.412717,0.140430,0.307555,0.333496,0.259837,0.249952,0.371869,...,0.359474,0.151826,0.425525,0.375087,0.277096,0.131071,0.230549,0.284602,0.434867,0.379136
15344,0.428091,0.000000,0.320105,0.389232,-0.040394,0.217663,0.283636,0.188385,0.227881,0.429392,...,0.182316,0.263529,0.424636,0.409913,0.352801,0.197683,0.253783,0.158278,0.438327,0.367678
29237,0.313774,0.320105,0.000000,0.312213,0.036993,0.289640,0.186558,0.135187,0.069235,0.397600,...,0.159255,0.186995,0.381735,0.374622,0.287239,0.050324,0.160204,0.258161,0.423712,0.333702
78410,0.412717,0.389232,0.312213,0.000000,0.072442,0.406765,0.240814,0.119701,0.340255,0.489007,...,0.302066,0.130979,0.478134,0.439999,0.377262,0.224233,0.269535,0.304163,0.477438,0.464281
83575,0.140430,-0.040394,0.036993,0.072442,0.000000,0.169615,0.146804,0.122561,0.177234,0.198552,...,0.290054,0.125784,0.189389,0.043521,0.169425,-0.057015,0.110793,0.073143,0.253698,0.154404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99845748,0.131071,0.197683,0.050324,0.224233,-0.057015,0.192945,0.236315,0.120183,0.227178,0.165379,...,0.238232,0.298568,0.430672,0.216807,0.253668,0.000000,0.152544,0.068944,0.345413,0.227731
99869481,0.230549,0.253783,0.160204,0.269535,0.110793,0.135921,0.115787,0.134768,0.271075,0.211613,...,0.090470,0.097052,0.383331,0.276408,0.092313,0.152544,0.000000,0.025657,0.257208,0.206810
99909118,0.284602,0.158278,0.258161,0.304163,0.073143,0.401549,0.425944,0.134234,0.024645,0.367457,...,0.362264,0.240941,0.222494,0.238388,0.330457,0.068944,0.025657,0.000000,0.303124,0.319565
99927639,0.434867,0.438327,0.423712,0.477438,0.253698,0.463754,0.427678,0.220933,0.347487,0.534573,...,0.288457,0.438649,0.583691,0.552196,0.352156,0.345413,0.257208,0.303124,0.000000,0.549119


In [12]:
neighboor_formation_static = coll.get_k_similar_users(correlation_matrix,2941,K = 200)

In [13]:
neighboor_formation_static

,14361993,56815255,43785832,14343424,7051818,50596997,74755744,14508560,49312804,29914016,...,30051866,29422130,39981635,64982269,88170732,86657631,14301263,55361439,38183203,23461723
2941,0.579992,0.562695,0.546981,0.546288,0.54027,0.539798,0.539466,0.53492,0.532015,0.524332,...,0.439487,0.439421,0.439269,0.439094,0.438964,0.438897,0.438824,0.43878,0.438319,0.438165


In [14]:
neighboor_formation_dynamic = coll.get_k_dynamic_similar_users(dense_user_item,2941,K = 200)

In [15]:
neighboor_formation_dynamic

,14361993,56815255,43785832,14343424,7051818,50596997,74755744,14508560,49312804,29914016,...,30051866,29422130,39981635,64982269,88170732,86657631,14301263,55361439,38183203,23461723
2941,0.579992,0.562695,0.546981,0.546288,0.54027,0.539798,0.539466,0.53492,0.532015,0.524332,...,0.439487,0.439421,0.439269,0.439094,0.438964,0.438897,0.438824,0.43878,0.438319,0.438165


## Popularity

Judge the popularity of user by the number of followers for its list

In [16]:
popularity = coll.get_popularity(lists_new,dense_user_item)

In [17]:
popularity

,popularity
2941,0.000000
15344,0.000000
29237,0.000000
78410,1.666667
83575,0.000000
...,...
99845748,0.111111
99869481,0.317460
99909118,18.777778
99927639,0.000000


In [18]:
k_popu = coll.get_k_popularity(popularity,neighboor_formation_dynamic)
k_popu

,14361993,56815255,43785832,14343424,7051818,50596997,74755744,14508560,49312804,29914016,...,30051866,29422130,39981635,64982269,88170732,86657631,14301263,55361439,38183203,23461723
popularity,0.0,0.007576,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.051136,0.0,0.0,0.013175,0.0,0.042208,0.556818


## Recommendation

In [19]:
popu_matrix = coll.get_popularity(lists_new,dense_user_item)
popu_matrix

,popularity
2941,0.000000
15344,0.000000
29237,0.000000
78410,1.666667
83575,0.000000
...,...
99845748,0.111111
99869481,0.317460
99909118,18.777778
99927639,0.000000


In [20]:
recomm_hybdrid = coll.hybdrid_RS(2941,dense_user_item,popu_matrix,neighboor_size=40,top_K=10,norm=True)
recomm_hybdrid.transpose()

100%|██████████| 2314/2314 [00:02<00:00, 980.23it/s] 


,302,304,367,406,441,510,668,1421,1505,9744
prediction,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.983053


In [21]:
recomm_pop = coll.popularity_RS(2941,dense_user_item,popu_matrix,neighboor_size=40,top_K=10,norm=True)
recomm_pop.transpose()

100%|██████████| 2314/2314 [00:02<00:00, 975.60it/s] 


,12,135,152,228,248,296,298,300,302,304
prediction,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0


In [22]:
recomm_classic = coll.classic_RS(2941,dense_user_item,neighboor_size=40,top_K=10,norm=True)
recomm_classic.transpose()

100%|██████████| 2314/2314 [00:02<00:00, 955.80it/s]


,406,441,510,668,791,1421,1505,9744,304,367
prediction,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.997246,4.974154,4.967247


In [23]:
recomm_trending = coll.trending_RS(2941,dense_user_item,popu_matrix,neighboor_size=40,top_K=10,norm=True)
recomm_trending.transpose()

100%|██████████| 2735/2735 [00:03<00:00, 841.29it/s] 


,436,21484,21836,25180,45235,162878,222,2296,17532,104606
prediction,5.0,5.0,5.0,5.0,5.0,5.0,4.999551,4.981263,4.980764,4.975991


(Too) many prediction are actually higher than 5 (it can happens if the average of U is higher than average of V, but V rated highly some movie in question -> around 6.1 value)

That's really annoying and didn't really found solution

So just redoing the formula without normalization

In [24]:
recomm_hybdrid_not = coll.hybdrid_RS(2941,dense_user_item,popu_matrix,neighboor_size=40,top_K=10)
recomm_hybdrid_not.transpose()

100%|██████████| 2314/2314 [00:02<00:00, 1082.34it/s]


,406,304,976,1421,370,2142,2091,510,353,823
prediction,4.901905,4.874165,4.855651,4.836609,4.835321,4.823942,4.816726,4.814931,4.806469,4.792606


In [25]:
recomm_pop_not = coll.popularity_RS(2941,dense_user_item,popu_matrix,neighboor_size=40,top_K=10)
recomm_pop_not.transpose()

100%|██████████| 2314/2314 [00:02<00:00, 1081.73it/s]


,683,723,951,1194,2142,2404,2480,2670,4014,8302
prediction,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0


In [26]:
recomm_classic_not = coll.classic_RS(2941,dense_user_item,neighboor_size=40,top_K=10)
recomm_classic_not.transpose()

100%|██████████| 2314/2314 [00:02<00:00, 1056.78it/s]


,406,441,304,510,1421,370,791,2091,976,353
prediction,4.934164,4.89475,4.843908,4.835631,4.829056,4.827579,4.825802,4.819454,4.809523,4.803953


In [27]:
recomm_trending_not = coll.trending_RS(2941,dense_user_item,popu_matrix,neighboor_size=40,top_K=10)
recomm_trending_not.transpose()

100%|██████████| 2735/2735 [00:02<00:00, 932.39it/s] 


,846,21836,170829,300,635,496,25180,697,1703,3062
prediction,4.972729,4.971559,4.965847,4.96424,4.956401,4.949133,4.942534,4.925877,4.924197,4.907823


## First complete try

In [2]:
#Pretty sexy :D
application()

Exception in Tkinter callback
Traceback (most recent call last):
  File "/home/vincent/miniconda3/envs/social/lib/python3.9/tkinter/__init__.py", line 1892, in __call__
    return self.func(*args)
  File "/home/vincent/unifr/sp22/social_media/SMA_Project/helper/application.py", line 16, in algorithm
    user = int(entry.get("1.0",'end-1c'))
ValueError: invalid literal for int() with base 10: 'Type a n2941ame'
100%|██████████| 2314/2314 [00:02<00:00, 989.32it/s] 


## Name test

In [12]:
dict_name = create_fake_identities(dense_user_item.index)
dict_name

{2941: 'Kayla Cunningham',
 15344: 'Ashley Ray',
 29237: 'Kevin Patton',
 78410: 'Charles Lamb',
 83575: 'Daniel Ingram',
 94978: 'Sarah Walker',
 99812: 'Heather Jones',
 114207: 'Brian Boyle',
 118841: 'Michelle Andrews',
 182362: 'Jodi Nichols',
 200681: 'Luis Hernandez',
 204061: 'Christopher Salas',
 225373: 'Kristopher Blanchard',
 230594: 'Jordan Rogers',
 234133: 'Joshua Kim',
 243873: 'Michele Mckinney',
 257723: 'Madison Wallace',
 282614: 'Tara Lang',
 325746: 'Michael Turner',
 373153: 'Richard Erickson',
 397936: 'Erin Cameron',
 400845: 'Dennis Jacobson',
 404311: 'Kathleen Michael',
 410815: 'Chelsea Gamble',
 426717: 'John Jackson',
 452234: 'Kathryn Camacho',
 457561: 'Jacqueline Morales',
 463202: 'Tiffany Vasquez',
 468202: 'David Paul',
 483726: 'Jason Robbins',
 509673: 'Daniel Blackburn',
 518214: 'Henry Fuentes',
 529544: 'Breanna Bowen',
 543478: 'Evan Jackson',
 557693: 'Kelly Roberts',
 561705: 'Matthew Solis',
 577908: 'Joshua Golden',
 636968: 'Anna Jones',


In [13]:
dict_name.keys()[dict_name.values().index("Kayla Cunningham")]

AttributeError: 'dict_values' object has no attribute 'index'

In [14]:
from faker import Faker

In [16]:
fake = Faker()

dir = {fake.name() : id for id in dense_user_item.index}

dir

{'Tara Henry': 2941,
 'Dr. Connor Rodriguez': 15344,
 'Nicholas Cain': 29237,
 'Susan Bennett': 78410,
 'Dr. William Bryan': 83575,
 'Courtney Miller': 94978,
 'Ronald Donovan': 99812,
 'Glenn Jones': 114207,
 'Krystal Olson': 118841,
 'Beth Ballard': 182362,
 'Tony Carter': 200681,
 'Mike Crawford': 204061,
 'Alan Adams': 225373,
 'Jamie Green': 230594,
 'Alex Sullivan': 234133,
 'Eric Jones': 51134155,
 'Paul Li': 257723,
 'Kristin Johnson': 282614,
 'Angela Reese': 325746,
 'Judy Mayo': 373153,
 'Kimberly Pham MD': 397936,
 'Ashley Dawson': 400845,
 'Tammy Schneider': 404311,
 'Nicole Patterson': 410815,
 'Angelica Jarvis MD': 426717,
 'Thomas Taylor': 452234,
 'Melissa Harrison': 457561,
 'Amber Garrett': 463202,
 'Anthony Diaz': 468202,
 'Maurice Griffin': 483726,
 'Ryan Madden': 509673,
 'Ivan Harrington': 518214,
 'Donald Hoffman': 529544,
 'Tony Hill': 543478,
 'Laura Scott': 557693,
 'Eric Wilkins': 561705,
 'Jason Sanchez': 577908,
 'Mark Baker': 636968,
 'William Miller': 63

In [25]:
list(dir.keys())[0]

False